The preprocessing module transforms gene expression, knockout, knockdown, and time series data into a graph format, which is then stored for utilization by the model.

#INSTALL

In [2]:
!pip install torch-geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 11.5 MB/s eta 0:00:00


In [3]:
pip install numpy-ml

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.9/239.9 kB 5.2 MB/s eta 0:00:00


#IMPORTS

In [1]:
from torch_geometric.data import Data
from torch_geometric.utils import convert
import torch
import pandas as pd
import torch_geometric.transforms as T
#import tensorflow as tf
import itertools
import numpy as np
import torch
from torch.nn import Linear
import seaborn as sns
from torch_geometric.nn import global_mean_pool
from torch_geometric.data import HeteroData

from torch_geometric.utils import negative_sampling

from torch_geometric.nn import ChebConv
from torch_geometric.nn import HypergraphConv
import torch.nn.functional as F

#FOR VISUALIZING GRAPH
import matplotlib.pyplot as plt
import networkx as nx
import random
from collections import Counter
from torch_geometric.utils import to_networkx


from sklearn.metrics import PrecisionRecallDisplay
from sklearn.metrics import top_k_accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import jaccard_score
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import roc_auc_score
from sklearn.metrics import RocCurveDisplay, roc_curve
from sklearn.metrics import DetCurveDisplay
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import auc
from sklearn.metrics import precision_recall_curve

In [2]:
import warnings
warnings.filterwarnings('ignore')
pd.options.display.max_rows=999
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


#DREAM4 Preprocessing

In [9]:
#Edge creation fucnction for the graph - reformatting the gene expression input
def convert_to_networkx(graph, n_sample=None):
    g = to_networkx(graph, node_attrs=["x"])
    y = graph.y.numpy()
    if n_sample is not None:
        sampled_nodes = random.sample(g.nodes, n_sample)
        g = g.subgraph(sampled_nodes)
        y = y[sampled_nodes]
    return g, y

def plot_graph(g, y):

    plt.figure(figsize=(9, 7))
    nx.draw_spring(g, node_size=30, arrows=False, node_color=y)
    plt.show()
def deviation_scores(knockoutfile,knockdownfile,edge_list): # passed things ar3e ecoli1_null,ecoli1_hetero

   edge_df = pd.DataFrame(edge_list, columns =['source', 'target'])
   wild_df = knockoutfile.iloc[0, :]
   knockoutfile=knockoutfile.iloc[1:, :]
   knockoutfile =knockoutfile.drop(['strain'], axis=1)
   ko_dev_df = pd.DataFrame(abs(knockoutfile.astype(float)-wild_df[1:].astype(float))) #knockoutfile.values-expanded_wild_df.values
   ko_dev_df = ko_dev_df.reset_index()
   ko_dev_df =ko_dev_df.drop(['index'], axis=1)

   knockdownfile=knockdownfile.iloc[1:, :]
   knockdownfile =knockdownfile.drop(['strain'], axis=1)
   kd_dev_df = pd.DataFrame(abs(knockdownfile.astype(float)-wild_df[1:].astype(float))) #knockdownfile.values-expanded_wild_df.values
   kd_dev_df = kd_dev_df.reset_index()
   kd_dev_df =kd_dev_df.drop(['index'], axis=1)

   edge_df['KO'] = 0
   edge_df['KD'] = 0

   ind=0
   for i in range(10):
     for j in range(10):
       if(i!=j):
         edge_df.loc[ind,'KO']=ko_dev_df.iat[i,j]
         edge_df.loc[ind,'KD']=kd_dev_df.iat[i,j]
         ind=ind+1
   return edge_df

def edge_label_creation(ecoli1_gold,edge_list):

   edge_df = pd.DataFrame(edge_list, columns =['source', 'target'])
   ecoli1_gold[0] = ecoli1_gold[0].str.replace('G', '')
   ecoli1_gold[1] = ecoli1_gold[1].str.replace('G', '')
   ecoli1_gold= ecoli1_gold.astype(int)
   ecoli1_gold[0] = ecoli1_gold[0] - 1
   ecoli1_gold[1] = ecoli1_gold[1] - 1

   edge_df['edge'] = 0
   for i in range(ecoli1_gold.shape[0]):
         r = ecoli1_gold.iat[i,0]
         c = ecoli1_gold.iat[i,1]
         idx= edge_df.loc[(edge_df['source'] == r) & (edge_df['target'] == c)].index
         edge_df.loc[idx,'edge']=ecoli1_gold.iat[i,2]
   return edge_df

def show_graph_stats(graph):
    print(f"Number of nodes: {graph.x.shape[0]}")
    print(f"Number of node features: {graph.x.shape[1]}")
    print(f"Number of edges: {graph.edge_index.shape[1]}")
    print(f"attributes: {graph.edge_attr}")


In [20]:
#Data preprocessing to graph
def data_preprocessing(folder_name,file_hetero,file_null,file_traject,file_gold,final_out,hidden1_channels,hidden2_channels,out_channels,num_layers):
  default_path="./data/DREAM4/DREAM4_InSilico_Size100/"#+folder_name+"/"+folder_name+"/"
  default_goldpath="./data/DREAM4/gold_std/"
  hetero = pd.read_csv(default_path+file_hetero,sep='\t')
  null = pd.read_csv(default_path+file_null,sep='\t')
  traject = pd.read_csv(default_path+file_traject,sep='\t')
  gold = pd.read_csv(default_goldpath+file_gold,sep='\t',header=None)

  wildtype_vals = hetero.loc[1, :].values.tolist()[:]
  print("wildtype_vals")
  print(wildtype_vals)
  hetero['id'] = hetero.index
  '''Gene id starts at 1 '''
  print("hetero")
  print(hetero)
  node_features = hetero[["id"]][0:-1]
  print("node_features")
  print(node_features)
  node_features['wildtype'] = wildtype_vals
  traj =traject.T.iloc[1:, 1:]
  traj=traj.reset_index()
  node_features = pd.concat([node_features, traj], axis=1)
  node_features=node_features.drop(['index'], axis=1)

  '''EXTRACTING EDGE FEATURES '''
  #KD, KO , SS
  edge_list = list(itertools.product(node_features["id"], repeat=2))

  '''EXTRACTING LABEL FEATURES '''
  edge_lab = edge_label_creation(gold,edge_list)

  null=null.iloc[1:, :]
  null = null.reset_index()
  null =null.drop(['index'], axis=1)
  null_list = null.values.flatten()
  print("null")
  print(len(null_list))
  hetero=hetero.iloc[1:, :]
  hetero = hetero.reset_index()
  hetero =hetero.drop(['index'], axis=1)
  hetero =hetero.drop(['id'], axis=1)
  hetero_list = hetero.values.flatten()
  print("hetero")
  print(len(hetero_list))



  #RANDOMLY SELECT TP AND TN edges to start the graph based on the percentage rates
  edge_lab.columns = ['s', 'd', 'edge']
  edge_lab['KO'] = null_list
  edge_lab['KD'] = hetero_list
  print("edge_lab")
  print(edge_lab.shape)
  edge_lab= edge_lab[edge_lab['edge'] == 1]

  src = edge_lab["s"].tolist()
  dst = edge_lab['d'].tolist()
  edge_lab_index = torch.tensor([src, dst])
  id = node_features["id"].tolist()
  wildtype = node_features["wildtype"].tolist()
  x = torch.tensor([id,wildtype]).T


  listfinal = []
  for column in node_features.columns:
    list1 = node_features[column].tolist()
    listfinal.append(list1)

  KO = edge_lab["KO"].tolist()
  KD = edge_lab["KD"].tolist()
  edscore = torch.tensor([KO,KD]).T

  basic_data = Data()
  basic_TS_data = Data()
  basic_aug_data = Data()
  basic_TS_aug_data = Data()


  basic_data.x = torch.tensor([id,wildtype]).T
  basic_data.edge_attr=edscore
  basic_data.edge_index = edge_lab_index

  basic_TS_data.x =torch.tensor(listfinal).T
  basic_TS_data.edge_attr=edscore
  basic_TS_data.edge_index = edge_lab_index

  G = convert.to_networkx(basic_data, to_undirected=True)
  pagerank = nx.algorithms.link_analysis.pagerank_alg.pagerank(G)
  clustering_coef = nx.algorithms.cluster.clustering(G)
  betweeness_centrality = nx.betweenness_centrality(G, k=50)
  degree = G.degree()

  pg_list =list(pagerank.values())
  clu_coef_list =list(clustering_coef.values())
  bet_cen_list =list(betweeness_centrality.values())

  deg_list =[x[1] for x in degree]

  basic_aug_data.x = torch.tensor([id,wildtype,pg_list,clu_coef_list,bet_cen_list,deg_list]).T
  basic_aug_data.edge_attr=edscore
  basic_aug_data.edge_index = edge_lab_index

  listfinal.append(pg_list)
  listfinal.append(clu_coef_list)
  listfinal.append(bet_cen_list)
  listfinal.append(deg_list)

  basic_TS_aug_data.x =torch.tensor(listfinal).T
  basic_TS_aug_data.edge_attr=edscore
  basic_TS_aug_data.edge_index = edge_lab_index
  print("basic_data")
  print(basic_data)
  print("basic_TS_data")
  print(basic_TS_data)
  print("basic_TS_aug_data")
  print(basic_TS_aug_data)
  print("basic_aug_data")
  print(basic_aug_data)
  return basic_data,basic_TS_data,basic_aug_data,basic_TS_aug_data



In [21]:
#Path for the files
gold_std = "./data/DREAM4/gold_std/"

InsilicoSize100_org  = {"Ecoli1":  ["InSilicoSize100","insilico_size100_1_knockdowns.tsv","insilico_size100_1_knockouts.tsv","insilico_size100_1_timeseries.tsv","DREAM4_GoldStandard_InSilico_Size100_1.tsv"],
                       "Ecoli2":  ["InSilicoSize100","insilico_size100_2_knockdowns.tsv","insilico_size100_2_knockouts.tsv","insilico_size100_2_timeseries.tsv","DREAM4_GoldStandard_InSilico_Size100_2.tsv"],
                       "Yeast1":  ["InSilicoSize100","insilico_size100_3_knockdowns.tsv","insilico_size100_3_knockouts.tsv","insilico_size100_3_timeseries.tsv","DREAM4_GoldStandard_InSilico_Size100_3.tsv"],
                       "Yeast2":  ["InSilicoSize100","insilico_size100_4_knockdowns.tsv","insilico_size100_4_knockouts.tsv","insilico_size100_4_timeseries.tsv","DREAM4_GoldStandard_InSilico_Size100_4.tsv"],
                       "Yeast3":  ["InSilicoSize100","insilico_size100_5_knockdowns.tsv","insilico_size100_5_knockouts.tsv","insilico_size100_5_timeseries.tsv","DREAM4_GoldStandard_InSilico_Size100_5.tsv"]
                        }

In [22]:
hidden1_channels=128
hidden2_channels=64
out_channels= 32
num_layers = 4
final_out = pd.DataFrame()
path = "./data/DREAM4/DREAM4_InSilico_Size100/"
for org, files in InsilicoSize100_org.items():
    basic_data,basic_TS_data,basic_aug_data,basic_TS_aug_data = data_preprocessing(files[0],files[1],files[2],files[3],files[4],final_out,hidden1_channels,hidden2_channels,out_channels,num_layers)
    torch. save(basic_data, path+"basic_data_"+org+"_"+files[0]+".pt")
    torch. save(basic_TS_data, path+"basic_TS_data_"+org+"_"+files[0]+".pt")
    torch. save(basic_aug_data, path+"basic_aug_data_"+org+"_"+files[0]+".pt")
    torch. save(basic_TS_aug_data, path+"basic_TS_aug_data_"+org+"_"+files[0]+".pt")

wildtype_vals
[0.1403573, 0.116043, 0.4982184, 0.2862524, 0.7081964, 0.037699, 0.6456384, 0.4476367, 0.0318306, 0.4442968, 0.0238948, 0.290782, 0.0326319, 0.0082785, 0.0087007, 0.1638587, 0.3807005, 0.4572849, 0.0382077, 0.0060735, 0.0457328, 0.1529319, 0.1343317, 0.1210139, 0.0798064, 0.6252963, 0.5300937, 0.2855325, 0.2104165, 0.4924795, 0.1824045, 0.0240991, 0.2441192, 0.1112913, 0.5829188, 0.6101525, 0.6275128, 0.1026203, 0.0524153, 0.7144504, 0.0295571, 0.6923681, 0.5879381, 0.5871904, 0.6965229, 0.6157876, 0.6422744, 0.6469524, 0.3772157, 0.2163583, 0.5234688, 0.6114584, 0.4512452, 0.2702531, 0.5086885, 0.4946782, 0.1577467, 0.238069, 0.1138431, 0.2476652, 0.0163729, 0.2866406, 0.4922897, 0.373348, 0.4634916, 0.4117426, 0.3038697, 0.3082255, 0.4509415, 0.0293569, 0.2779169, 0.0273554, 0.0497445, 0.101345, 0.4442863, 0.5563797, 0.0169423, 0.0605418, 0.1512659, 0.0280181, 0.3677316, 0.1158194, 0.6521404, 0.1296269, 0.5309754, 0.2242765, 0.1140824, 0.0423087, 0.0624452, 0.6042533, 0

ValueError: Length of values (100) does not match length of index (99)

In [8]:
import os

# 경로 설정
path = "./data/DREAM4/DREAM4_InSilico_Size100/"

# 경로가 존재하는지 확인
if os.path.exists(path):
    print(f"The path exists: {path}")
    # 경로 안의 파일 목록 출력
    files = os.listdir(path)
    print(f"Files in the directory: {files}")
else:
    print(f"The path does not exist: {path}")


The path exists: ./data/DREAM4/DREAM4_InSilico_Size100/
Files in the directory: ['insilico_size100_5', 'insilico_size100_2', 'insilico_size100_4_timeseries.tsv', 'insilico_size100_5_knockouts.tsv', 'insilico_size100_3', 'insilico_size100_1_timeseries.tsv', 'insilico_size100_5_knockdowns.tsv', 'insilico_size100_2_timeseries.tsv', 'insilico_size100_2_knockouts.tsv', 'insilico_size100_1', 'insilico_size100_1_knockouts.tsv', 'insilico_size100_3_knockouts.tsv', 'insilico_size100_3_knockdowns.tsv', 'insilico_size100_5_timeseries.tsv', 'insilico_size100_2_knockdowns.tsv', 'insilico_size100_3_timeseries.tsv', 'insilico_size100_4_knockouts.tsv', 'insilico_size100_4', 'insilico_size100_1_knockdowns.tsv', 'insilico_size100_4_knockdowns.tsv']


#DREAM5 Preprocessing

In [6]:
#Edge creation fucnction for the graph - reformatting the gene expression input
def edge_label_creation(ecoli1_gold,edge_list):
   edge_df = pd.DataFrame(edge_list, columns =['source', 'target'])
   ecoli1_gold[0] = ecoli1_gold[0].str.replace('G', '')
   ecoli1_gold[1] = ecoli1_gold[1].str.replace('G', '')
   ecoli1_gold= ecoli1_gold.astype(int)
   ecoli1_gold[0] = ecoli1_gold[0] - 1
   ecoli1_gold[1] = ecoli1_gold[1] - 1
   edge_df['source'] = edge_df['source'] - 1
   edge_df['target'] = edge_df['target'] - 1
   edge_df['edge'] = 0

   ecoli1_gold.columns =['source', 'target', 'edge']

   edge_df['source'] = edge_df['source'].astype(int)
   edge_df['target'] = edge_df['target'].astype(int)
   ecoli1_gold['edge'] = ecoli1_gold['edge'].astype(int)
   edge_df = pd.merge(edge_df, ecoli1_gold,  how='outer', on=['source','target'])#, right_on = ['source','target'])
   edge_df =edge_df.drop(['edge_x'], axis=1)

   edge_df.fillna(0,inplace=True)
   edge_df['edge_y'] = edge_df['edge_y'].astype(int)
   c = ['x', 'y']
   return edge_df

def show_graph_stats(graph):
    print(f"Number of nodes: {graph.x.shape[0]}")
    print(f"Number of node features: {graph.x.shape[1]}")
    print(f"Number of edges: {graph.edge_index.shape[1]}")
    print(f"attributes: {graph.edge_attr}")

In [7]:
#Data preprocessing to graph
def data_preprocessing(folder_name,file_hetero,file_gold,final_out,hidden1_channels,hidden2_channels,out_channels,num_layers):
  default_path=path
  default_goldpath=gold_std
  hetero = pd.read_csv(default_path+file_hetero,sep='\t')
  gold = pd.read_csv(default_goldpath+file_gold,sep='\t',header=None)
  hetero=hetero.T

  hetero['id'] = hetero.index
  # '''Gene id starts at 1 '''
  hetero['id'] = hetero['id'].str.replace('G', '')
  hetero['id']= hetero['id'].astype(int)
  hetero['id'] = hetero['id'] - 1
  print("hetero")
  print(hetero)

  '''EXTRACTING EDGE FEATURES '''
  edge_list = list(itertools.product(hetero["id"], repeat=2))
  '''EXTRACTING LABEL FEATURES '''
  edge_lab = edge_label_creation(gold,edge_list)
  edge_lab.columns = ['s', 'd', 'edge']
  edge_lab= edge_lab[edge_lab['edge'] == 1]
  src = edge_lab["s"].tolist()
  dst = edge_lab['d'].tolist()
  edge_lab_index = torch.tensor([src, dst])

  '''without id onlly 42.7 percent accruacy'''
  listfinal = []
  for column in hetero.columns:
    list1 = hetero[column].tolist()
    listfinal.append(list1)

  basic_data = Data()
  basic_aug_data = Data()

  print("listfinal")
  print(listfinal)
  basic_data.x =torch.tensor(listfinal).T
  basic_data.edge_index = edge_lab_index

  G = convert.to_networkx(basic_data, to_undirected=True)
  pagerank = nx.algorithms.link_analysis.pagerank_alg.pagerank(G)
  clustering_coef = nx.algorithms.cluster.clustering(G)
  betweeness_centrality = nx.betweenness_centrality(G, k=50)
  degree = G.degree()

  pg_list =list(pagerank.values())
  clu_coef_list =list(clustering_coef.values())
  bet_cen_list =list(betweeness_centrality.values())

  deg_list =[x[1] for x in degree]
  listfinal.append(pg_list)
  listfinal.append(clu_coef_list)
  listfinal.append(bet_cen_list)
  listfinal.append(deg_list)

  basic_aug_data.x =torch.tensor(listfinal).T
  basic_aug_data.edge_index = edge_lab_index
  print("basic_data")
  print(basic_data)
  print("basic_aug_data")
  print(basic_aug_data)
  return basic_data,basic_aug_data



In [8]:
#Path for the files
path = "/content/drive/MyDrive/Colab Notebooks/data/DREAM5/training data/"
gold_std = "/content/drive/MyDrive/Colab Notebooks/data/DREAM5/gold/test data/"

organism  = {"Insilico":  ["InSilicoSize100","net1_expression_data.tsv","DREAM5_NetworkInference_GoldStandard_Network1 - in silico.tsv"],
                        # "Saure":  ["InSilicoSize100","net2_expression_data.tsv","DREAM5_NetworkInference_GoldStandard_Network2 - S. aureus.txt"],
                        "Ecoli":  ["InSilicoSize100","net3_expression_data.tsv","DREAM5_NetworkInference_GoldStandard_Network3 - E. coli.tsv"],
                        "Scere":  ["InSilicoSize100","net4_expression_data.tsv","DREAM5_NetworkInference_GoldStandard_Network4 - S. cerevisiae.tsv"]
                            }

In [11]:
#main run
hidden1_channels=128
hidden2_channels=64
out_channels= 32
num_layers = 4
final_out = pd.DataFrame()

for org, files in organism.items():
    basic_data,basic_aug_data = data_preprocessing(files[0],files[1],files[2],final_out,hidden1_channels,hidden2_channels,out_channels,num_layers)
    torch. save(basic_data, path+"basic_data_"+org+"_"+files[0]+".pt")
    torch. save(basic_aug_data, path+"basic_aug_data_"+org+"_"+files[0]+".pt")

hetero
              0         1         2         3         4         5         6  \
G1     0.425447  0.442400  1.056847  1.117226  0.971068  1.139386  1.064869   
G2     0.017829  0.050525  0.208454  0.003001  0.001056  0.122047  0.140508   
G3     0.907989  0.869368  0.467448  0.317654  0.354651  0.402465  0.481763   
G4     0.448247  0.445851  0.505077  0.387204  0.474532  0.348436  0.474857   
G5     0.172324  0.173311  0.244883  0.253792  0.207718  0.168614  0.182643   
...         ...       ...       ...       ...       ...       ...       ...   
G1639  0.822844  0.915617  0.349069  0.449126  0.512270  0.754407  0.659339   
G1640  0.304483  0.317507  0.042310  0.125197  0.261410  0.064153  0.051364   
G1641  0.319917  0.238074  0.165208  0.000047  0.000156  0.040764  0.035758   
G1642  0.364280  0.509130  0.952178  0.878127  0.883981  0.766373  0.655370   
G1643  0.765945  0.691403  0.678781  0.566691  0.646715  0.725356  0.748289   

              7         8         9  ...    

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



basic_data
Data(x=[1643, 806], edge_index=[2, 4012])
basic_aug_data
Data(x=[1643, 810], edge_index=[2, 4012])
hetero
            0       1        2        3        4        5        6       7  \
G1     7.1151  7.2840   6.6090   6.4579   6.6933   6.6903   6.9693  7.4548   
G2     9.3293  9.2212   8.6310   8.7221   8.6554   8.4474   8.3946  8.5799   
G3     9.5997  9.4961  10.2730  10.1430  10.0250  10.5120  10.5870  9.4264   
G4     6.9998  6.8697   7.0817   7.0796   7.0943   7.3964   7.2210  7.4652   
G5     7.4955  7.4101   7.5102   7.4146   7.5755   7.4651   7.6309  7.7096   
...       ...     ...      ...      ...      ...      ...      ...     ...   
G4507  7.6695  7.9183   7.8730   7.8176   7.6769   7.8061   7.7681  8.8945   
G4508  8.2989  8.2432   7.8048   7.7709   7.8688   8.2781   8.2339  8.7803   
G4509  6.4199  6.7346   6.6771   6.9012   6.7569   6.5984   6.2633  6.6815   
G4510  7.3635  7.3835   8.3439   8.2427   8.2085   7.5338   7.6780  9.4176   
G4511  9.9146  8.6026   8

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



basic_data
Data(x=[4511, 806], edge_index=[2, 2066])
basic_aug_data
Data(x=[4511, 810], edge_index=[2, 2066])
hetero
             0        1        2        3        4        5        6        7  \
G1      8.3066   8.2621   8.2446   9.1807   9.1068   8.9059   8.7250   8.6023   
G2      9.7335   9.8441   9.5841   9.8570   9.7898  10.1570  10.1960   9.8746   
G3     10.3450  10.0650  10.1500  10.1890  10.1500  10.8880  10.2570  10.2710   
G4      9.1199   9.2828  10.5620   9.7156   9.3488   9.2581   9.5359   9.8865   
G5     10.3480  10.3880  10.3130  10.5940  10.4410  10.6500  10.7960  10.5390   
...        ...      ...      ...      ...      ...      ...      ...      ...   
G5946  14.4970  14.4490  14.4320  14.2070  14.3660  14.2920  14.3240  14.3950   
G5947   7.4510   7.7653   7.8711   7.9300   7.9784   8.3869   8.7664   8.5995   
G5948  11.0010  10.9860  10.3720  11.4650  11.4400  11.1640  10.6470  10.9220   
G5949   7.7354   8.0687   8.1065   8.0246   7.7537   7.9353   7.7506   7.

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



basic_data
Data(x=[5950, 537], edge_index=[2, 3940])
basic_aug_data
Data(x=[5950, 541], edge_index=[2, 3940])
